# Análisis Exploratorio de Datos (EDA)
## Detección de Lavado de Activos en Colombia

**Objetivo Específico 1 (OE1):** Caracterizar y analizar los patrones de comportamiento asociados al lavado de activos mediante el análisis de datos históricos de transacciones, identificando las variables predictoras más relevantes.

Este notebook explora el dataset sintético de transacciones para:
- Comprender la distribución de clases (fraude vs. normal)
- Analizar patrones en montos, tipos de transacción y balances
- Identificar correlaciones entre variables
- Detectar features discriminantes para modelado

In [ ]:
import sys
sys.path.append('../..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

# Carga datos
df = pd.read_csv('../../data/synthetic/aml_colombia_synthetic.csv')
print(f"📊 Dataset cargado: {df.shape}")
print(f"   Período: {df['step'].min()} - {df['step'].max()} días")
print(f"   Transacciones: {len(df):,}")

## 1. Información General del Dataset

In [ ]:
# Información básica
print("=" * 60)
print("INFORMACIÓN DEL DATASET")
print("=" * 60)
print(df.info())
print("\n")
print("Primeras filas:")
df.head(10)

In [ ]:
# Estadísticas descriptivas
print("=" * 60)
print("ESTADÍSTICAS DESCRIPTIVAS")
print("=" * 60)
df.describe()

## 2. Distribución de Clases (Fraude vs. Normal)

In [ ]:
# Distribución de fraude
fraud_counts = df['isFraud'].value_counts()
fraud_pct = df['isFraud'].value_counts(normalize=True) * 100

print("=" * 60)
print("DISTRIBUCIÓN DE CLASES")
print("=" * 60)
print(f"Normal (0):  {fraud_counts[0]:,} ({fraud_pct[0]:.2f}%)")
print(f"Fraude (1):  {fraud_counts[1]:,} ({fraud_pct[1]:.2f}%)")
print(f"\nRatio desbalanceo: 1:{fraud_counts[0]/fraud_counts[1]:.1f}")

# Visualización
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Barras
fraud_counts.plot(kind='bar', ax=axes[0], color=['#2ecc71', '#e74c3c'])
axes[0].set_title('Distribución de Clases (Conteo)', fontsize=14, weight='bold')
axes[0].set_xlabel('Clase')
axes[0].set_ylabel('Número de Transacciones')
axes[0].set_xticklabels(['Normal', 'Fraude'], rotation=0)
axes[0].grid(axis='y', alpha=0.3)

# Pie
axes[1].pie(fraud_counts, labels=['Normal', 'Fraude'], autopct='%1.2f%%',
           colors=['#2ecc71', '#e74c3c'], startangle=90)
axes[1].set_title('Proporción de Clases', fontsize=14, weight='bold')

plt.tight_layout()
plt.savefig('../../reports/figures/distribucion_clases.png', dpi=300)
plt.show()

## 3. Distribución de Tipos de Transacción

In [ ]:
# Tipos de transacción
print("=" * 60)
print("TIPOS DE TRANSACCIÓN")
print("=" * 60)
print(df['type'].value_counts())
print("\nPorcentajes:")
print(df['type'].value_counts(normalize=True) * 100)

# Fraude por tipo
print("\n" + "=" * 60)
print("FRAUDE POR TIPO DE TRANSACCIÓN")
print("=" * 60)
fraud_by_type = df.groupby('type')['isFraud'].agg(['sum', 'mean', 'count'])
fraud_by_type.columns = ['Total_Fraude', 'Tasa_Fraude', 'Total_Transacciones']
fraud_by_type['Tasa_Fraude'] = fraud_by_type['Tasa_Fraude'] * 100
fraud_by_type = fraud_by_type.sort_values('Tasa_Fraude', ascending=False)
print(fraud_by_type)

# Visualización
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# Conteo por tipo
df['type'].value_counts().plot(kind='bar', ax=axes[0], color='skyblue')
axes[0].set_title('Transacciones por Tipo', fontsize=14, weight='bold')
axes[0].set_xlabel('Tipo de Transacción')
axes[0].set_ylabel('Número de Transacciones')
axes[0].grid(axis='y', alpha=0.3)
plt.setp(axes[0].xaxis.get_majorticklabels(), rotation=45, ha='right')

# Tasa de fraude por tipo
fraud_by_type['Tasa_Fraude'].plot(kind='bar', ax=axes[1], color='coral')
axes[1].set_title('Tasa de Fraude por Tipo (%)', fontsize=14, weight='bold')
axes[1].set_xlabel('Tipo de Transacción')
axes[1].set_ylabel('Porcentaje de Fraude')
axes[1].grid(axis='y', alpha=0.3)
plt.setp(axes[1].xaxis.get_majorticklabels(), rotation=45, ha='right')

plt.tight_layout()
plt.savefig('../../reports/figures/fraude_por_tipo.png', dpi=300)
plt.show()

## 4. Análisis de Montos de Transacción

In [ ]:
# Estadísticas de montos
print("=" * 60)
print("ESTADÍSTICAS DE MONTOS (COP)")
print("=" * 60)
print("\nTodas las transacciones:")
print(df['amount'].describe())
print("\nNormal vs. Fraude:")
print(df.groupby('isFraud')['amount'].describe())

# Visualización
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Histograma general (log scale)
axes[0, 0].hist(df['amount'], bins=50, color='skyblue', edgecolor='black', alpha=0.7)
axes[0, 0].set_title('Distribución de Montos (Todas)', fontsize=12, weight='bold')
axes[0, 0].set_xlabel('Monto (COP)')
axes[0, 0].set_ylabel('Frecuencia')
axes[0, 0].set_yscale('log')
axes[0, 0].grid(alpha=0.3)

# Histograma por clase
df[df['isFraud']==0]['amount'].hist(bins=50, ax=axes[0, 1], alpha=0.6, label='Normal', color='green')
df[df['isFraud']==1]['amount'].hist(bins=50, ax=axes[0, 1], alpha=0.6, label='Fraude', color='red')
axes[0, 1].set_title('Distribución de Montos por Clase', fontsize=12, weight='bold')
axes[0, 1].set_xlabel('Monto (COP)')
axes[0, 1].set_ylabel('Frecuencia')
axes[0, 1].legend()
axes[0, 1].set_yscale('log')
axes[0, 1].grid(alpha=0.3)

# Boxplot por clase
df.boxplot(column='amount', by='isFraud', ax=axes[1, 0])
axes[1, 0].set_title('Boxplot de Montos por Clase', fontsize=12, weight='bold')
axes[1, 0].set_xlabel('Clase (0=Normal, 1=Fraude)')
axes[1, 0].set_ylabel('Monto (COP)')
plt.sca(axes[1, 0])
plt.xticks([1, 2], ['Normal', 'Fraude'])

# Violinplot
sns.violinplot(data=df, x='isFraud', y='amount', ax=axes[1, 1])
axes[1, 1].set_title('Violin Plot de Montos', fontsize=12, weight='bold')
axes[1, 1].set_xlabel('Clase')
axes[1, 1].set_ylabel('Monto (COP)')
axes[1, 1].set_xticklabels(['Normal', 'Fraude'])

plt.tight_layout()
plt.savefig('../../reports/figures/analisis_montos.png', dpi=300)
plt.show()

## 5. Análisis de Balances

In [ ]:
# Balances por clase
print("=" * 60)
print("ANÁLISIS DE BALANCES")
print("=" * 60)
print("\nBalance Origen (oldbalanceOrg):")
print(df.groupby('isFraud')['oldbalanceOrg'].describe())
print("\nBalance Destino (oldbalanceDest):")
print(df.groupby('isFraud')['oldbalanceDest'].describe())

# Visualización
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Old Balance Origen
df[df['isFraud']==0]['oldbalanceOrg'].hist(bins=50, ax=axes[0, 0], alpha=0.6, label='Normal', color='green')
df[df['isFraud']==1]['oldbalanceOrg'].hist(bins=50, ax=axes[0, 0], alpha=0.6, label='Fraude', color='red')
axes[0, 0].set_title('Balance Inicial Origen', fontsize=12, weight='bold')
axes[0, 0].set_xlabel('Balance (COP)')
axes[0, 0].set_ylabel('Frecuencia')
axes[0, 0].legend()
axes[0, 0].grid(alpha=0.3)

# New Balance Origen
df[df['isFraud']==0]['newbalanceOrig'].hist(bins=50, ax=axes[0, 1], alpha=0.6, label='Normal', color='green')
df[df['isFraud']==1]['newbalanceOrig'].hist(bins=50, ax=axes[0, 1], alpha=0.6, label='Fraude', color='red')
axes[0, 1].set_title('Balance Final Origen', fontsize=12, weight='bold')
axes[0, 1].set_xlabel('Balance (COP)')
axes[0, 1].set_ylabel('Frecuencia')
axes[0, 1].legend()
axes[0, 1].grid(alpha=0.3)

# Old Balance Destino
df[df['isFraud']==0]['oldbalanceDest'].hist(bins=50, ax=axes[1, 0], alpha=0.6, label='Normal', color='green')
df[df['isFraud']==1]['oldbalanceDest'].hist(bins=50, ax=axes[1, 0], alpha=0.6, label='Fraude', color='red')
axes[1, 0].set_title('Balance Inicial Destino', fontsize=12, weight='bold')
axes[1, 0].set_xlabel('Balance (COP)')
axes[1, 0].set_ylabel('Frecuencia')
axes[1, 0].legend()
axes[1, 0].grid(alpha=0.3)

# New Balance Destino
df[df['isFraud']==0]['newbalanceDest'].hist(bins=50, ax=axes[1, 1], alpha=0.6, label='Normal', color='green')
df[df['isFraud']==1]['newbalanceDest'].hist(bins=50, ax=axes[1, 1], alpha=0.6, label='Fraude', color='red')
axes[1, 1].set_title('Balance Final Destino', fontsize=12, weight='bold')
axes[1, 1].set_xlabel('Balance (COP)')
axes[1, 1].set_ylabel('Frecuencia')
axes[1, 1].legend()
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.savefig('../../reports/figures/analisis_balances.png', dpi=300)
plt.show()

## 6. Matriz de Correlación

In [ ]:
# Codifica tipo de transacción
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df_encoded = df.copy()
df_encoded['type_encoded'] = le.fit_transform(df['type'])

# Features numéricas para correlación
numeric_features = ['step', 'amount', 'oldbalanceOrg', 'newbalanceOrig',
                   'oldbalanceDest', 'newbalanceDest', 'type_encoded', 'isFraud']

# Matriz de correlación
corr_matrix = df_encoded[numeric_features].corr()

print("=" * 60)
print("CORRELACIÓN CON FRAUDE")
print("=" * 60)
fraud_corr = corr_matrix['isFraud'].sort_values(ascending=False)
print(fraud_corr)

# Heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
           square=True, linewidths=0.5, cbar_kws={"shrink": 0.8})
plt.title('Matriz de Correlación - Features vs Fraude', fontsize=14, weight='bold', pad=20)
plt.tight_layout()
plt.savefig('../../reports/figures/matriz_correlacion.png', dpi=300)
plt.show()

## 7. Variables más Discriminantes

Identificación de las features con mayor capacidad de discriminación entre fraude y normal.

In [ ]:
# Top features correlacionadas con fraude
print("=" * 60)
print("TOP VARIABLES PREDICTORAS (correlación absoluta)")
print("=" * 60)
top_features = fraud_corr.abs().sort_values(ascending=False)[1:8]  # Excluye isFraud mismo
print(top_features)

# Visualización de top features
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.flatten()

top_cols = ['amount', 'oldbalanceOrg', 'newbalanceOrig', 
           'oldbalanceDest', 'newbalanceDest', 'type_encoded']

for i, col in enumerate(top_cols):
    if i < len(axes):
        df_encoded.boxplot(column=col, by='isFraud', ax=axes[i])
        axes[i].set_title(f'{col}', fontsize=11, weight='bold')
        axes[i].set_xlabel('Clase (0=Normal, 1=Fraude)')
        axes[i].set_ylabel('Valor')
        plt.sca(axes[i])
        plt.xticks([1, 2], ['Normal', 'Fraude'])

plt.suptitle('Distribución de Top Features por Clase', fontsize=16, weight='bold', y=1.02)
plt.tight_layout()
plt.savefig('../../reports/figures/top_features.png', dpi=300)
plt.show()

## 8. Conclusiones del EDA

### Hallazgos Clave:

1. **Desbalanceo de Clases**: ~1% de transacciones son fraude (1:99 ratio)
   - Requiere técnicas de balanceo (SMOTE, class weights)
   
2. **Tipos de Transacción Sospechosos**:
   - CASH_OUT y TRANSFER tienen mayor tasa de fraude
   - PAYMENT y DEBIT son mayormente legítimos
   
3. **Montos**:
   - Fraudes tienen montos significativamente más altos
   - Distribución lognormal con cola pesada en fraudes
   
4. **Balances**:
   - Fraudes frecuentemente vacían cuentas (newbalanceOrig ≈ 0)
   - Balances finales cercanos a cero son altamente sospechosos
   
5. **Features Discriminantes** (mayor correlación con fraude):
   - `amount`: Montos altos
   - `newbalanceOrig`: Balance final origen
   - `oldbalanceOrg`: Balance inicial origen
   - `type_encoded`: Tipo de transacción
   
### Recomendaciones para Modelado:

- ✅ Usar **SMOTE** para balancear clases
- ✅ Crear **features derivadas**: ratios de balance, flags de montos altos/redondos
- ✅ **Validación cruzada estratificada** para preservar distribución de clases
- ✅ Métricas enfocadas en fraude: **Recall, F1-score, AUC-ROC**
- ✅ Modelos ensemble: **XGBoost, Random Forest** para capturar interacciones no lineales